# Assignment 10

### 1.复习上课内容

### 2. 回答一下理论题目

#### 1. What is independent assumption in Naive bayes ?

Ans:
Naive Bayes is so called because the independence assumptions we have just made are indeed very naive for a model of natural language. The conditional independence assumption states that features are independent of each other given the class.   
Definition: X is conditionally independent of Y given Z, if the probability distribution governing X is independent of the value of Y, given the value of Z   
<center>$\forall$(i,j,k) P(X=$x_{i}$|$Y=y_{j}$,Z=$z_{k}$) = P(X=$x_{i}$|Z=$z_{k}$) </center>
Which we often write 
<center>P(X|Y,Z) = P(X|Z)</center>

#### 2. What is MAP(maximum a posterior) and ML(maximum likelihood) ?

In [ ]:
Ans:


#### 3. What is support vector in SVM?

#### 4. What is the intuition behind SVM ?

#### 5. Shortly describ what 'random' means in random forest ?

#### 6. What cariterion does XGBoost use to find the best split point in a tree ?

### 3. Practial part

##### Problem description: In this part you are going to build a classifier to detect if a piece of news is published by the Xinhua news agency (新华社）.

#### Hints:

###### 1. Firstly, you have to come up with a way to represent the news. (Vectorize the sentence, you can find different ways to do so online)  

###### 2. Secondly,  pick a machine learning algorithm that you think is suitable for this task

### Congratulations! You have completed all assignments in this week. The question below is optional. If you still have time, why don't try it out.

## Option:

#### Try differnt machine learning algorithms with different combinations of parameters in the practical part, and compare their performances (Better use some visualization techiniques).